# Análisis de los diferentes modelos

In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
from dotenv import load_dotenv
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

In [ ]:
load_dotenv()

In [ ]:
data_samples = {
    'question': ['When was the first super bowl?', 'Who won the most super bowls?'],
    'answer': ['The first superbowl was held on Jan 15, 1967', 'The most super bowls have been won by The New England Patriots'],
    'contexts' : [
        ['The First AFL–NFL World Championship Game was an American football game played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles,'],
        ['The Green Bay Packers...Green Bay, Wisconsin.','The Packers compete...Football Conference']
    ],
    'ground_truth': ['The first superbowl was held on January 15, 1967', 'The New England Patriots have won the Super Bowl a record six times']
}

dataset = Dataset.from_dict(data_samples)

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
generation_config = {
    'max_new_tokens': 1024,
    'temperature': 0.0001,
    'top_p': 0.95,
    'do_sample': True,
    'repetition_penalty': 1.15
}

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

In [ ]:
llm = HuggingFacePipeline(
    pipeline=text_pipeline,
    model_kwargs={"temperature": 0}
)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
score = evaluate(
    dataset,
    metrics=[faithfulness,answer_correctness],
    llm=llm,
    embeddings=embeddings
)

df_score = score.to_pandas()
print(df_score)